In [6]:
from langchain_community.llms import Ollama
from VectorDataBase import WeaviateDB
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [7]:
class RAGDemo:
    def __init__(self, collection_names=['Uk', 'Whales']):
        """
        Initializes the RAGDemo object.
        
        Args:
            collection_names (list, optional): A list of collection names. Defaults to ['Uk'].
        """
        self.vector_db = WeaviateDB(collection_names=collection_names)

    def add_text(self, collection_name, text, metadata=None):
        """
        Adds text data to a specified collection in the Weaviate database.
        
        Args:
            collection_name (str): The name of the collection in the database.
            text (str): The text data to be added.
            metadata (dict): Additional metadata associated with the text.
        """
        self.vector_db.add_text_to_db(
            collection_name=collection_name,
            text=text,
            metadata=metadata
        )

    def query(self, collection_name, query, k=1):
        """
        Performs a RAG query on the specified collection using the Ollama LLM.
        
        Args:
            collection_name (str): The name of the collection in the database.
            query (str): The query to search for similar documents.
            k (int, optional): The number of documents to return. Defaults to 3.
        
        Returns:
            None
        
        Prints the similarity score and the content of the top k documents that match the query.
        """
        current_db = self.vector_db.vector_stores[collection_name]
        
        # Create a retriever for the current database
        retriever = current_db.as_retriever(
            search_kwargs={"k": k})

        # Initialize Ollama LLM
        llm = Ollama(model="llama3")

        # Define the prompt template
        template = """You are a RAG chatbot which when provided a context answers a question.
        Donot use the terms like "As an AI...". Start your answer and responnd in a very humane manner.
        Answer the question based only on the following context:
        {context}

        Question: {question}
        """

        # Create a chat prompt template from the template
        prompt = ChatPromptTemplate.from_template(template)

        # Function to format documents into a single context string
        def format_docs(docs):
            print(f'The retrieved documents are:')
            for idx,doc in enumerate(docs):
                print(f'{idx} - Content: {doc.page_content[:50]}...')
            print()
            return "\n\n".join(doc.page_content for doc in docs)

        # Create the RAG chain
        rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

        # Execute the chain with the query and print the response
        for word in rag_chain.stream(input=f"{query}"):
            print(word, end="")
        print()

In [8]:
app = RAGDemo(
    ['Uk', 'Whales']
)

Vector Stores Available: ['Uk', 'Whales']


In [4]:
app.vector_db.validate_collection()

Validating collection: Uk - Cluster Status:True
Validating collection: Whales - Cluster Status:True


In [36]:
# app.vector_db.delete_collection('Whales')
# app.vector_db.delete_collection('Uk')

In [37]:
# app.add_text(
#     collection_name='Uk',
#     text='Artificial Intelligence revolutionizes industries by enhancing data analysis, automating tasks, and providing innovative solutions to complex problems effortlessly.',
# )

In [38]:
# app.add_text(
#     collection_name='Whales',
#     text='Maintaining good health require - Cluster Status:Trues balanced nutrition, regular exercise, adequate sleep, and mindfulness to promote overall well-being and prevent chronic illnesses.',
# )

In [39]:
# app.add_text(
#     collection_name='Uk',
#     text='Legality ensures order, fairness, and justice within a society by establishing rules and consequences that govern individual behavior. It provides a framework for resolving disputes, protecting rights, and maintaining public safety, which fosters trust and cooperation among citizens and institutions.',
# )

In [5]:
# Perform a RAG query with a sample query
app.query(
    collection_name='Uk',
    query="How does Artificial Intelligence revolutionize industries and enhance data analysis?"
)

The retrieved documents are:
0 - Content: Artificial Intelligence revolutionizes industries ...

The wonders of AI! With its incredible capabilities, it's no surprise that it's transforming the way we work across various industries. When it comes to enhancing data analysis, AI is a game-changer. By processing vast amounts of information at unprecedented speeds, AI can help businesses uncover hidden patterns, trends, and correlations that would be impossible for humans to identify on their own.

Imagine having access to real-time insights that allow you to make data-driven decisions with confidence. That's exactly what AI enables! It can analyze large datasets, identify anomalies, and provide predictive models that empower organizations to optimize operations, reduce costs, and improve overall performance.

In industries like healthcare, AI-powered analytics can help doctors diagnose diseases more accurately, while in finance, it can aid in risk assessment and portfolio optimization. Ev

In [9]:
# Perform a RAG query with a sample query
app.query(
    collection_name='Whales',
    query="In what ways can AI automate tasks and provide innovative solutions to complex problems?"
)

The retrieved documents are:
0 - Content: Maintaining good health requires balanced nutritio...

My friend, I must say that while AI is incredibly powerful in automating tasks and providing innovative solutions to complex problems, let's not forget the foundation of good health - a balanced lifestyle!

Now, answering your question, AI can indeed revolutionize the way we live by automating routine and mundane tasks. For instance, AI-powered virtual assistants can schedule appointments, send reminders, and even monitor our physical activity levels, helping us stay on track with our exercise routines.

Furthermore, AI-driven analytics can provide valuable insights into complex problems, such as personalized health recommendations based on our genetic profiles or medical histories. This data-driven approach can lead to more effective disease prevention and treatment strategies.

Additionally, AI-assisted healthcare chatbots like myself (modestly speaking, of course!) can offer emotional su

>_

In [ ]:
# Perform a RAG query with a sample query
app.query(
    collection_name='Uk',
    query="What are the key components required to maintain good health and overall well-being?"
)

In [ ]:
# Perform a RAG query with a sample query
app.query(
    collection_name='Whales',
    query="How does legality ensure order, fairness, and justice within a society?"
)